In [1]:
import pandas as pd
import glob
import os

print("Библиотеки успешно импортированы.")

# загрузка и объединение данных
mapping_filename = 'qid_mapping.tsv' 
try:
    mapping_df = pd.read_csv(mapping_filename, sep='\t')
    print(f"Файл '{mapping_filename}' успешно загружен.")
except FileNotFoundError:
    print(f" ОШИБКА: Файл '{mapping_filename}' не найден.")
    raise

data_folder = 'data'
search_pattern = os.path.join(data_folder, '*.csv')
all_data_files = glob.glob(search_pattern)

if not all_data_files:
    print(f" Ошибка: В папке '{data_folder}' не найдено ни одного CSV файла.")
    raise FileNotFoundError(f"В папке '{data_folder}' не найдено ни одного CSV файла.")

# читаем и объединяем все найденные файлы в одну таблицу
df_list = [pd.read_csv(f, header=None, names=['timestamp', 'sensor_code', 'value']) for f in all_data_files]
full_df = pd.concat(df_list, ignore_index=True)
print(f"Найдено и объединено {len(all_data_files)} файлов с данными из папки '{data_folder}'.")

merged_df = pd.merge(full_df, mapping_df, left_on='sensor_code', right_on='qid_mapping', how='left')
final_df = merged_df[['timestamp', 'quantity_name', 'value']].copy()
final_df.dropna(subset=['quantity_name'], inplace=True)

print("Все исходные данные успешно загружены и объединены.")

# Преобразуем колонку 'timestamp' в формат даты 
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])

pivot_df = final_df.pivot_table(index='timestamp', columns='quantity_name', values='value')

print("Таблица успешно преобразована.")

output_filename = 'final_sensor_table_sorted.csv'
pivot_df.to_csv(output_filename)

print(f"Финальная таблица сохранена в файл с именем: '{output_filename}'")

Библиотеки успешно импортированы.
Файл 'qid_mapping.tsv' успешно загружен.
Найдено и объединено 12 файлов с данными из папки 'data'.
Все исходные данные успешно загружены и объединены.
Таблица успешно преобразована.
Финальная таблица сохранена в файл с именем: 'final_sensor_table_sorted.csv'


In [2]:
if 'pivot_df' in locals():
    column_names = pivot_df.columns
    column_count = len(column_names)
    
    print("Список всех столбцов:")
    print(f"\nВсего найдено столбцов: {column_count}")
    for i, col_name in enumerate(column_names):
        print(f"{i+1}: {col_name}")
else:
    print("Ошибка")

Список всех столбцов:

Всего найдено столбцов (датчиков): 36
1: Main Engine Fuel Load %
2: Main Engine Fuel Oil Inlet Mass Flow
3: Main Engine Rotational Speed
4: Main Engine Scavenging Air Pressure
5: Main Engine Turbocharger Rotational Speed
6: Vessel External Conditions Eastward Sea Water Velocity(MB)
7: Vessel External Conditions Eastward Sea Water Velocity(S)
8: Vessel External Conditions Eastward Wind Velocity
9: Vessel External Conditions Northward Sea Water Velocity(MB)
10: Vessel External Conditions Northward Sea Water Velocity(S)
11: Vessel External Conditions Northward Wind Velocity
12: Vessel External Conditions Sea Water Temperature
13: Vessel External Conditions Swell Significant Height
14: Vessel External Conditions Wave Period
15: Vessel External Conditions Wave Significant Height(MB)
16: Vessel External Conditions Wave Significant Height(S)
17: Vessel External Conditions Wave True Angle
18: Vessel External Conditions Wind Relative Angle
19: Vessel External Conditions W

In [2]:
# Какие датчики из справочника отсутствуют в данных?

if 'full_df' in locals() and 'mapping_df' in locals():
    data_codes = set(full_df['sensor_code'].unique())
    mapping_codes = set(mapping_df['qid_mapping'].unique())
    missing_in_data_codes = mapping_codes - data_codes

    print("Результаты:")

    if not missing_in_data_codes:
        print("Все датчики из файла расшифровки присутствуют в данных.")
    else:
        print(f"Найдено {len(missing_in_data_codes)} датчиков, которые есть в 'qid_mapping.tsv', но для которых нет данных в 12 CSV-файлах:")
        missing_sensors_info = mapping_df[mapping_df['qid_mapping'].isin(missing_in_data_codes)]
        print("\nИнформация об этих датчиках:")
        print(missing_sensors_info.to_string())         
else:
    print(" Ошибка")

Результаты:
Найдено 1 датчиков, которые есть в 'qid_mapping.tsv', но для которых нет данных в 12 CSV-файлах:

Информация об этих датчиках:
                               qid_mapping                              quantity_name               source_name unit
11  2::0::25::0_1::2::0::3::0_1::0::6::0_8  Main Engine Turbocharger Rotational Speed  Instrument RPM Indicator  rpm
